In [1]:
import cv2
import face_recognition as fr
import os
import numpy as np

In [2]:
# Essa função carrega várias imagens de uma pasta e as armazena em uma lista
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Verifica se o arquivo é uma imagem
            img_path = os.path.join(folder, filename)
            img = fr.load_image_file(img_path)  # Carrega a imagem
            images.append((filename, img))  # Adiciona a imagem e seu nome à lista
    return images

In [3]:
# pré-processa imagens da lista, converte para RGB, detecta rostos nelas, 
# e pega as os encondings (codificações faciais) e os armazena em listas
def preprocess_images(images):
    processed_images = []
    encodings = []
    for filename, img in images:
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)#convertendo em rgb
        face_locations = fr.face_locations(rgb_img)
        if face_locations:
            face_encoding = fr.face_encodings(rgb_img, face_locations)[0] #pegando as codificações faciais
            processed_images.append((filename, rgb_img, face_locations[0]))
            encodings.append(face_encoding)
    return processed_images, encodings

In [4]:
#  exibe imagens em uma janela,converte de RGB para BGR
# define que a imagem fecha ao apertar esc ou aguardando 10 segundos 
def show_images(images):
    for filename, img in images:
        # Convertendo de RGB para BGR 
        bgr_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imshow(filename, bgr_img)
        # Aguarde 10 segundos entre mostrar as imagens ou até pressionar esc
        if cv2.waitKey(10000) & 0xFF == 27: 
            break
    cv2.destroyAllWindows()


In [5]:
# redimensiona as imagens e coloca um tamanho máximo
def resize_image(image, max_size):
    height, width = image.shape[:2]
    scaling_factor = min(max_size / float(height), max_size / float(width))
    new_size = (int(width * scaling_factor), int(height * scaling_factor))
    resized_image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
    return resized_image

In [6]:
# coloca as imagens em listas
recognition_folder = r'D:\FIAP\2024\1_Semestre\AI ENGENEERING, COGNITIVE AND SEMANTIC COMPUTATION & IOT\CP\KenAI\Reconhecimento_pessoa\images\imagens'
test_folder = r'D:\FIAP\2024\1_Semestre\AI ENGENEERING, COGNITIVE AND SEMANTIC COMPUTATION & IOT\CP\KenAI\Reconhecimento_pessoa\images\Test'

# Carregar e pré-processar imagens
recognition_images = load_images_from_folder(recognition_folder)
test_images = load_images_from_folder(test_folder)

# chama a função preprocess_images
processed_recognition_images, recognition_encodings = preprocess_images(recognition_images)
processed_test_images, test_encodings = preprocess_images(test_images)

# Tamanho máximo das imagens
max_image_size = 400

In [7]:
# Comparar o rosto reconhecido com o rosto das fotos
for test_filename, test_img, test_face_loc in processed_test_images:
    test_face_encoding = fr.face_encodings(test_img, [test_face_loc])[0]
    matches = fr.compare_faces(recognition_encodings, test_face_encoding)
    face_distances = fr.face_distance(recognition_encodings, test_face_encoding)
    
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
        name = "LARISSA"
        #firula apenas
        color = (0, 255, 0)  # Verde
        font = 3
        tamanho = 100
    else:
        name = "OUTRA PESSOA"
         #mais firula
        color = (0, 0, 255)  # Vermelho
        font = 0.8
        tamanho = 40

    # Desenhar o retângulo ao redor do rosto detectado
    cv2.rectangle(test_img, (test_face_loc[3], test_face_loc[0]), (test_face_loc[1], test_face_loc[2]), color, 6)
    
    # Colocar o texto logo abaixo do retângulo
    text_position = (test_face_loc[3], test_face_loc[2] + tamanho)
    cv2.putText(test_img, name, text_position, cv2.FONT_HERSHEY_DUPLEX, font, color, 2, cv2.LINE_AA)

    # Redimensionar a imagem se necessário
    resized_img = resize_image(test_img, max_image_size)
    
    # Mostrar imagem de teste com a anotação
    cv2.imshow('Result', resized_img)
    key = cv2.waitKey(0) & 0xFF
    if key == 27:  # Press 'Esc' to move to the next image
        continue

# Liberar todas as janelas do OpenCV
cv2.destroyAllWindows()